<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.callbacks import FileCallbackHandler

from loguru import logger

logfile = "output_03.log"
with open(logfile, "a") as f:
    # We want to start each run with a clean log file
    f.truncate(0)

def get_diff_in_result(text: str):
    _, after = text.split("```diff")
    return after.split("```")[0]


#logger.add(logfile, colorize=True, enqueue=True)
#handler = FileCallbackHandler(logfile)

template = """You are an excellent enterprise architect who has an extensive
background in helping companies rewrite their legacy Java EE applications to Quarkus.

You will read a user's problem along with examples of how they have solved a problem in the past.
The past examples will be presented in format of a summary of the issue along with source code of 
that point in time along with the updated source code when the problem is fixed

You will then write Quarkus code to solve their current problem.
You will output the results in the form a diff which can be applied via 'git apply'.

Example #1 Issue: {example1_issue}

Example #1 Original Source Code:
```java
{example1_original_code}
```

Example #1 Solved Source Code:
```java
{example1_solved_code}
```

Current Issue: 
{current_issue}

Current Issue Original Source Code: 
```java
{current_issue_original_code}
```

Your job is to look at the 'Current Issue' and the 'Current Issue Original Source Code' 
and rewrite the 'Current Issue Original Source Code' so the 'Current Issue' is solved 
in a manner similar to how 'Example #1 Original Source Code' was rewritten to 
'Example #1 Solved Source Code' 

Think through the changes you will make and explain each step of the process.
If you are unsure of what changes is needed please state you are unsure and ask 
for clarification to help you.

When you are done explaining the reasoning for each change, write the updated 
Quarkus source code in the form of a diff from the original file in the Markdown format of:

```diff
....
```"""

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4-1106-preview")
prompt = PromptTemplate.from_template(template)
#chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler], verbose=True)
chain = LLMChain(llm=llm, prompt=prompt)

Turn debug on so we can see more of what langchain is doing

In [ ]:
from langchain.globals import set_debug
set_debug(False)

Let's get an example together from a Java EE app that has been migrated to Quarkus

We will be using the repo from:  
* https://github.com/mathianasj/eap-coolstore-monolith/tree/quarkus-migration

There is a script to help with fetching the repo at: 
* https://github.com/jwmatthews/kyma_poc/blob/main/data/fetch.sh

In [ ]:
from kyma_poc.scm import GitDiff

# We are taking 2 similar files in this repo and will using them as an example 
# to see quality of diff we can get from the LLM
example_git_path = "../data/coolstuff-quarkus"
gd = GitDiff(example_git_path)

example_1_file = "src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java"
current_file = "src/main/java/com/redhat/coolstore/service/InventoryNotificationMDB.java"

example_1_file_commits = gd.get_commits_for_file(example_1_file)
current_file_commits = gd.get_commits_for_file(current_file)

example_1_original = gd.get_file_contents(example_1_file, example_1_file_commits[-1])
example_1_solved = gd.get_file_contents(example_1_file, example_1_file_commits[0])
current_original = gd.get_file_contents(current_file, example_1_file_commits[-1])

expected_diff = gd.get_patch_for_file(example_1_file_commits[-1], example_1_file_commits[0], example_1_file)

template_args = {
"example1_issue": "Something",
"example1_original_code": example_1_original,
"example1_solved_code": example_1_solved,
"current_issue": "Something",
"current_issue_original_code": current_original
}

formatted_prompt = template.format(**template_args)
print(f"Formatted Prompt:\n{formatted_prompt}")

Formatted Prompt:
You are an excellent enterprise architect who has an extensive
background in helping companies rewrite their legacy Java EE applications to Quarkus.

You will read a user's problem along with examples of how they have solved a problem in the past.
The past examples will be presented in format of a summary of the issue along with source code of 
that point in time along with the updated source code when the problem is fixed

You will then write Quarkus code to solve their current problem.
You will output the results in the form a diff which can be applied via 'git apply'.

Example #1 Issue: Something

Example #1 Original Source Code:
```java
package com.redhat.coolstore.service;

import javax.ejb.ActivationConfigProperty;
import javax.ejb.MessageDriven;
import javax.inject.Inject;
import javax.jms.JMSException;
import javax.jms.Message;
import javax.jms.MessageListener;
import javax.jms.TextMessage;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.

Lets run the chain and see it work

In [ ]:
result = chain.run(template_args)
print(f"Chain finished running\nResult:\n")
print(f"{result}")

result_diff = get_diff_in_result(result)

markdown_log = "example_03b_gpt-4-1106-preview.md"
with open(markdown_log, "w") as f:
    f.write(f"# Example #3\n")
    f.write(f"## Prompt:\n")
    f.write(f"{formatted_prompt}\n")
    f.write(f"## Result:\n")
    f.write(f"{result}\n")
    f.write(f"### Actual Diff:\n")
    f.write(f"```\n{result_diff}\n```\n")
    f.write(f"### Expected Diff:\n")
    f.write(f"```\n{expected_diff}\n```\n")

Chain finished running
Result:

To rewrite the `InventoryNotificationMDB` class from the legacy Java EE application to Quarkus, we will follow a similar approach to the one used in Example #1. Here are the steps we will take:

1. Replace the `MessageListener` interface and JMS-specific code with MicroProfile Reactive Messaging, which is a Quarkus-compatible way to handle messaging.

2. Remove the JNDI and JMS setup code, as Quarkus will manage the connection to the messaging system.

3. Use the `@Incoming` annotation to specify the channel from which messages will be received.

4. Use the `@Blocking` annotation to indicate that the processing of the message is blocking and should not be done on the I/O thread.

5. Add the `@ApplicationScoped` annotation to define the bean as application-scoped.

6. Remove the `init` and `close` methods as they are no longer needed. Quarkus will manage the lifecycle of the messaging components.

7. Replace the `Logger` injection with a Quarkus-compatibl